## Corona-News Scraper from Tagesschau.de

In [15]:
import re
import requests
from bs4 import BeautifulSoup

import datetime
import pandas as pd

In [16]:
todays_date = datetime.datetime.now()
date_stamp = str(todays_date.date())

In [17]:
r = requests.get('https://www.tagesschau.de/')
soup = BeautifulSoup(r.text, "html5lib")

### Parse Tagesschau.de homepage for new articles

In [18]:
headlines = []
dachzeilen = []
teasers = []
links = []
texts = []
a = soup.find_all('a')
for item in a:
    if item.find('p', class_='dachzeile'):
        links.append(item.get('href'))
        texts.append(item.get_text())

In [19]:
# check the first text / link
print(texts[0])

print(links[0])



Coronavirus-Pandemie
+++ New Yorks Gouverneur gegen Trump +++
Andrew Cuomo hat im Streit über eine mögliche Lockerung von Corona-Maßnahmen den US-Präsidenten zurechtgewiesen. NRW bereitet sich auf eine schrittweise Öffnung von Schulen vor. Das kündigten zwei Minister an. Alle Entwicklungen im Liveblog.
| mehr


/newsticker/liveblog-coronavirus-dienstag-101.html


### Check which articles on the homepage are about the Corona virus

In [20]:
# corona, covid19, coronavirus
corona_links = []
corona_texts = []
for i in range(len(texts)):
    if ("corona" in texts[i].lower()) | ("covid" in texts[i].lower()):
        if links[i].startswith("/"):
            corona_links.append(links[i])
            corona_texts.append(texts[i])

In [21]:
print(corona_links)
print("")
print(corona_texts)

['/newsticker/liveblog-coronavirus-dienstag-101.html', '/inland/corona-schule-105.html', '/inland/quarantaen-alte-menschen-corona-101.html', '/inland/rki-pk-101.html', '/wirtschaft/corona-krise-iwf-101.html', '/wirtschaft/coronavirus-milchbauern-101.html', '/inland/stichwort-leopoldina-101.html', '/investigativ/swr/coronakrise-gestrandete-pakistan-101.html', '/ausland/tuerkei-freilassungen-101.html', '/wirtschaft/amazon-265.html', '/ausland/coronavirus-karten-101.html']

['\nCoronavirus-Pandemie\n+++ New Yorks Gouverneur gegen Trump +++\nAndrew Cuomo hat im Streit über eine mögliche Lockerung von Corona-Maßnahmen den US-Präsidenten zurechtgewiesen. NRW bereitet sich auf eine schrittweise Öffnung von Schulen vor. Das kündigten zwei Minister an. Alle Entwicklungen im Liveblog.\n|\xa0mehr\n\n', '\nCorona-Maßnahmen\nStreitfall Schule\nRegelmäßiges Lehren und Lernen, mehr Raum für berufstätige Eltern - der Wunsch nach einer Rückkehr in den Schulalltag ist groß. Doch wie kann die sicher geli

### Get full article for each news about corona virus

In [22]:
corona_articles = []
for link in corona_links:
    full_link = "https://www.tagesschau.de" + link
    print(full_link)
    html = requests.get(full_link)
    sub_soup = BeautifulSoup(html.text, "html5lib")
    article = sub_soup.find_all('p', class_='text small')
    paragraphs = []
    for i in range(len(article)):
        paragraphs.append(article[i].get_text())
    # unifying the paragraphs
    final_article = " ".join(paragraphs)
    corona_articles.append(final_article)
    

https://www.tagesschau.de/newsticker/liveblog-coronavirus-dienstag-101.html
https://www.tagesschau.de/inland/corona-schule-105.html
https://www.tagesschau.de/inland/quarantaen-alte-menschen-corona-101.html
https://www.tagesschau.de/inland/rki-pk-101.html
https://www.tagesschau.de/wirtschaft/corona-krise-iwf-101.html
https://www.tagesschau.de/wirtschaft/coronavirus-milchbauern-101.html
https://www.tagesschau.de/inland/stichwort-leopoldina-101.html
https://www.tagesschau.de/investigativ/swr/coronakrise-gestrandete-pakistan-101.html
https://www.tagesschau.de/ausland/tuerkei-freilassungen-101.html
https://www.tagesschau.de/wirtschaft/amazon-265.html
https://www.tagesschau.de/ausland/coronavirus-karten-101.html


In [23]:
print(corona_articles)

['Andrew Cuomo hat im Streit über eine mögliche Lockerung von Corona-Maßnahmen den US-Präsidenten zurechtgewiesen. NRW bereitet sich auf eine schrittweise Öffnung von Schulen vor. Das kündigten zwei Minister an.  ', 'Regelmäßiges Lehren und Lernen, mehr Raum für berufstätige Eltern - der Wunsch nach einer Rückkehr in den Schulalltag ist groß. Doch wie kann die sicher gelingen? Die Experten sind sich uneins.  Seit Wochen sind die Schulen in Deutschland geschlossen. Eltern stemmen den Heimunterricht und ihre Büroarbeit oftmals parallel in den eigenen vier Wänden. Die Schüler bangen um ihre Abschlussprüfungen und lernen unter erschwerten Bedingungen. Die Sehnsucht nach einer Rückkehr zu einem geregelten Schulalltag ist groß, doch wie die gelingen kann, darüber streiten die Experten. \n  Eine Schulöffnung nur in kleinen Schritten und in den Hauptfächern, die jüngeren Abschlussjahrgänge zuerst - das empfehlen die Wissenschaftler der Leopoldina in ihrem vielfach beachteten Bericht. "Da die J

In [24]:
len(corona_articles)

11

### Make into CSV file

In [25]:
columns = ["date", "url", "article", "resort"]
df = pd.DataFrame(columns=columns)
df["url"] = corona_links
df["article"] = corona_articles
df["date"] = todays_date
df["resort"] = df["url"].apply(lambda x: x.split("/")[1])

In [26]:
df.head()

,date,url,article,resort
0,2020-04-14 18:50:51.964342,/newsticker/liveblog-coronavirus-dienstag-101....,Andrew Cuomo hat im Streit über eine mögliche ...,newsticker
1,2020-04-14 18:50:51.964342,/inland/corona-schule-105.html,"Regelmäßiges Lehren und Lernen, mehr Raum für ...",inland
2,2020-04-14 18:50:51.964342,/inland/quarantaen-alte-menschen-corona-101.html,Viele ältere Menschen fühlen sich jetzt bereit...,inland
3,2020-04-14 18:50:51.964342,/inland/rki-pk-101.html,"RKI-Präsident Wieler sieht ""einige positive Te...",inland
4,2020-04-14 18:50:51.964342,/wirtschaft/corona-krise-iwf-101.html,Die Welt steht wegen der Corona-Pandemie laut ...,wirtschaft


### Export CSV

In [27]:
df.to_csv("data/" + date_stamp)